In [11]:
pip install transformers evaluate datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer # baseline code용 tfidef vectorirzer(또는 tokenizer)
from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import display, HTML

from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer # 사용하고자 하는 모델, 토크나이저 적용시 필요
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, ClassLabel

from tqdm.auto import tqdm # process bar 표시용

import warnings
warnings.filterwarnings(action='ignore') 

import urllib.request

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device  # pt 코드에서 필요함

device(type='cuda')

In [48]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
train_data = pd.read_table('ratings_train.txt')
val_data = pd.read_table('ratings_test.txt')

In [8]:
pretrained_model = 'klue/roberta-small' 
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [61]:
train_data = train_data.dropna()
val_data = val_data.dropna()

In [62]:
from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(train_data)
vds = Dataset.from_pandas(val_data)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

In [63]:
# 데이터 tokenize
def preprocess_function(examples):
    return tokenizer(examples["document"], truncation=True,  return_token_type_ids=False,)

tokenized_datasets = ds.map(preprocess_function, batched=True)



  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [64]:
# loss type define
import evaluate
accuracy = evaluate.load("accuracy")

In [65]:
# 문장을 7개 type으로 분류
# loss(accuracy) 적용
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "부정", 1: "긍정" }
label2id = {"부정":0, "긍정":1}

In [66]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'cl

In [67]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id, __index_level_0__. If document, id, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 149995
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9375
  Number of trainable parameters = 68092418
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.252800,0.262874,0.896574


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id, __index_level_0__. If document, id, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 49997
  Batch size = 16
Saving model checkpoint to my_model/checkpoint-9375
Configuration saved in my_model/checkpoint-9375/config.json
Model weights saved in my_model/checkpoint-9375/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-9375/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-9375/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from my_model/checkpoint-9375 (score: 0.26287415623664856).


TrainOutput(global_step=9375, training_loss=0.2945893351236979, metrics={'train_runtime': 1084.8246, 'train_samples_per_second': 138.267, 'train_steps_per_second': 8.642, 'total_flos': 2451297569486820.0, 'train_loss': 0.2945893351236979, 'epoch': 1.0})

In [68]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id, __index_level_0__. If document, id, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 49997
  Batch size = 16


{'eval_loss': 0.26287415623664856,
 'eval_accuracy': 0.8965737944276656,
 'eval_runtime': 91.099,
 'eval_samples_per_second': 548.82,
 'eval_steps_per_second': 34.303,
 'epoch': 1.0}

In [72]:
text =''
inputs =  tokenizer(text, truncation=True,  return_token_type_ids=False, return_tensors="pt").to(device)
model.eval()
outputs = model(**inputs)
predictions = outputs.logits.argmax(-1)
id2label[predictions.cpu().numpy()[0]]

'부정'

In [70]:
outputs.logits

tensor([[-0.3022,  0.3655]], device='cuda:0', grad_fn=<AddmmBackward0>)